In [1]:
# Framework:
#
# 1. Individual stock:
#   1.0. GENERAL INPUT:
#      - position_date_1
#      - end_date
#      - ticker
#   1.1. Historical price plot: plot_price_stock()
#   1.2. Parameter estimates: plot_parameters_stock()
#   1.3. VaR/ES: plot_risk_stock()
#      - EXTRA INPUT: see VaR/ES input below
# 2. Portfolio: 
#   1.0. GENERAL INPUT:
#      - position_date
#      - end_date 
#      - tickers_string
#      - weight_string
#   1.1. Historical price plot: plot_price_stock()
#   1.2. Parameter estimates: plot_parameters_stock()
#   1.3. VaR/ES: plot_risk_stock()
#      - EXTRA INPUT: see VaR/ES input below
#
# Questions:
#
# 1. Imp vol for options available on Yahoo?
# 2. If need method to take parameters??
# 3. 你多注意一下 global/local variables 的问题
#   因为毕竟我们现在都是在这里把全部cell跑了一遍，所有的variable都是存下来的
#   不确定在网页上只运行一部分的时候会不会出问题
#


In [2]:
# Assumptions:
#
# 1. For all portfolio calculation, assume portfolio follows GBM, not using underlyings to simulate because
#    we don't know how to determin correlations for >2 positions
# 2. For historical VaR/ES, we use relative price changes instead of absolute price chagnes (log return)
# 3. The system doesn't support portfolio analysis with short positions
# 

In [3]:
# TODO:
#
# 1. Sigma plot
# 2. Plot legend outside plot area
# 3. Options as positions (hw10)
# 4. Backtest (hw11)
#

In [4]:
# import packages

import pandas_datareader.data as web
import datetime
import pandas as pd
import numpy as np
import scipy.stats as stat
import dateutil.relativedelta
import sys

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from __future__ import division

In [5]:
##########################################################################################
################################### GENERAL METHODS ######################################
##########################################################################################

In [6]:
# Price plot
def plot_price(price, length):
    data = price[:length]
    output_notebook()    
    plot = figure(width=600, height=400, title = "Historical Prices", x_axis_type="datetime")
    plot.line(data.index, data)
    plot.title.text_font_size = '12pt'
    show(plot)

In [7]:
# Calculate estimated parameters for GBM based on x year (in days) rolling windows
def gbm_est(prices, window_days):
    rtn = -np.diff(np.log(prices))
    rtnsq = rtn * rtn
    mubar = list(reversed(np.convolve(rtn, np.ones((window_days,))/window_days, mode='valid')))
    x2bar = list(reversed(np.convolve(rtnsq, np.ones((window_days,))/window_days, mode='valid')))
    var = x2bar - np.square(mubar)
    sigmabar = np.sqrt(np.maximum(var, np.zeros(len(var))))
    sigma = sigmabar / np.sqrt(1/252)
    mu = np.array(mubar)*252 + np.square(sigma)/2
    return rtn, mu, sigma, np.array(mubar), sigmabar

In [8]:
# TODO:
#   - subplot??
#   - fancy!!!!!! (legend, color, title, ...)

def plot_parameters(price):
    rtn_2, mu_2, sigma_2, mubar_2, sigmabar_2 = gbm_est(price, 2*252)
    rtn_5, mu_5, sigma_5, mubar_5, sigmabar_5 = gbm_est(price, 5*252)
    rtn_10, mu_10, sigma_10, mubar_10, sigmabar_10 = gbm_est(price, 10*252)
    length = min(len(mu_2), len(mu_5), len(mu_10), len(sigma_2), len(sigma_5), len(sigma_10))
    mu = pd.DataFrame({'Mu_2': mu_2[:length], 'Mu_5': mu_5[:length], 'Mu_10': mu_10[:length]}, 
                      index = price.index[:length])
    sigma = pd.DataFrame({'Sigma_2': sigma_2[:length], 'Sigma_5': sigma_5[:length], 'Sigma_10': sigma_10[:length]}, 
                         index = price.index[:length])
    output_notebook()    
    plot = figure(width=600, height=400, title = "Mu", x_axis_type="datetime")
    plot.line(mu.index, mu['Mu_2'], legend = '2-year roling window')
    plot.line(mu.index, mu['Mu_5'], color = 'green', legend = '5-year roling window')
    plot.line(mu.index, mu['Mu_10'], color = 'orange', legend = '10-year roling window')
    plot.legend.location = 'bottom_left'
    plot.title.text_font_size = '12pt'
    show(plot)

In [9]:
# Calculate VaR and ES using parametric method

def parametric(v0, mu, sigma, VaR_prob, ES_prob, t):
    VaR = v0 - v0 * np.exp(sigma * np.sqrt(t) * stat.norm.ppf(1-VaR_prob) + (mu - np.square(sigma)/2) * t)
    ES = v0 * (1 - np.array(stat.norm.cdf(stat.norm.ppf(1-ES_prob) - np.sqrt(t)*sigma)) * np.array(np.exp(mu*t)/(1-ES_prob)))
    return VaR, ES

In [10]:
# Calculate VaR and ES using historical method

def historical(v0, price, VaR_prob, ES_prob, window_days, horizon_days):
    npaths = window_days - horizon_days
    ntrials = len(price) - window_days
    price_log = np.log(price)
    return_xdays = np.array(price_log[:(len(price_log)-horizon_days)]) - np.array(price_log[5:])
    price_res = v0 * np.exp(return_xdays)
    scenarios = np.zeros(shape=(npaths,ntrials))
    for i in range(ntrials):
        scenarios[0:npaths,i] = price_res[i:i+npaths]
    scenarios_sorted = np.sort(scenarios, axis=0)
    VaR = v0 - scenarios_sorted[np.ceil((1-VaR_prob)*npaths).astype(int) - 1]
    ES = v0 - np.mean(scenarios_sorted[0:(np.ceil((1-ES_prob)*npaths).astype(int))], axis=0)
    return VaR, ES

In [11]:
# Calculate VaR and ES using Monte Carlo method

def monte_carlo(v0, price, mu, sigma, VaR_prob, ES_prob, window_days, horizon):
    npaths = 5000
    ntrials = len(price) - window_days
    p1 = np.zeros(shape=(npaths,ntrials))
    for i in range(ntrials):
        tv = np.ones(shape =(npaths,1))*horizon
        bm = np.sqrt(horizon) * np.random.randn(npaths,1)
        y = v0 * np.exp(sigma[i] * bm - (mu[i] + sigma[i]*sigma[i]/2) * tv)
        p1[:,i] = y[:,0]
    p2 = np.sort(p1,axis = 0)
    VaR = v0 - p2[np.ceil((1-VaR_prob)*npaths).astype(int) - 1]
    ES = v0 - np.mean(p2[0:(np.ceil((1-ES_prob)*npaths).astype(int))], axis=0)
    return VaR, ES  

In [12]:
# VaR/ES plot

def plot_risk(v0, price, VaR_prob, ES_prob, method, window, horizon, plot_length):
    if method == 'Parametric Method':
        rtn, mu, sigma, mubar, sigmabar = gbm_est(price, window*252)
        VaR, ES = parametric(v0, mu, sigma, VaR_prob, ES_prob, horizon)
    elif method == 'Historical Method':
        VaR, ES = historical(v0, price, VaR_prob, ES_prob, window*252, horizon*252)
    elif method == 'Monte Carlo':
        rtn, mu, sigma, mubar, sigmabar = gbm_est(price, window*252)
        VaR, ES = monte_carlo(v0, price, mu, sigma, VaR_prob, ES_prob, window*252, horizon)
    else:
        sys.exit('Error!')
        
    length = min(len(VaR), len(ES), plot_length)
    VaR_ES = pd.DataFrame({'VaR': VaR[:length], 'ES': ES[:length]}, index = price.index[:plot_length])
    output_notebook()    
    plot = figure(width=600, height=400, title = "VaR/ES", x_axis_type="datetime")
    plot.line(VaR_ES.index, VaR_ES['VaR'], color = 'orange', legend = 'VaR')
    plot.line(VaR_ES.index, VaR_ES['ES'], color = 'green', legend = 'ES')
    plot.legend.location = 'top_left'
    plot.title.text_font_size = '12pt'
    show(plot)

In [13]:
##########################################################################################
################################ PART 1: STOCK ANALYSIS ##################################
##########################################################################################

In [14]:
### Set up input

position_date_1 = datetime.datetime(2010, 12, 1)   # first day of plot
end_date_1 = datetime.datetime(2016, 12, 1)        # pick up a date for testing, may remove later and set today as default
ticker_1 = "AAPL"                                  # Stock ticker (string)

### VaR/ES input
v0_1 = 10000                                         # initial investment
VaR_prob_1 = 0.99
ES_prob_1 = 0.975
window_1 = 2                                         # using 2 years historical data for estimation
horizon_days_1 = 5                                   # 5 days VaR/ES
method_1 = 'Parametric Method'

# Not input, prepared for further analysis
window_days_1 = window_1 * 252                         # convert to days
horizon_1 = horizon_days_1/252                         # convert to years
start_date_1 = position_date_1 - dateutil.relativedelta.relativedelta(years = 10)      # fetch data starting from start_date
price_1 = web.DataReader(ticker_1, 'yahoo', start_date_1, end_date_1)['Adj Close'].sort_index(ascending = False)  
plot_length_1 = len(price_1[price_1.index >= position_date_1])


In [15]:
def plot_price_stock():
    plot_price(price_1, plot_length_1)

In [16]:
def plot_parameters_stock():
    plot_parameters(price_1)

In [17]:
def plot_risk_stock():
    plot_risk(v0_1, price_1, VaR_prob_1, ES_prob_1, method_1, window_1, horizon_1, plot_length_1)

In [18]:
##########################################################################################
######################## PART 2: PORTFOLIO ANALYSIS ######################################
##########################################################################################

In [19]:
### Set up input

position_date = datetime.datetime(2010, 12, 1)   # first day of plot
end_date = datetime.datetime(2016, 12, 1)        # pick up a date for testing, may remove later and set today as default
tickers_string = "AAPL,AMZN"   # a string; as an example, tickers separated by comma.
weight_string = "0.5,0.5"
    
### VaR/ES input
v0 = 10000                                         # initial investment
VaR_prob = 0.99
ES_prob = 0.975
window = 2                                         # using 2 years historical data for estimation
horizon_days = 5                                   # 5 days VaR/ES
method = 'Parametric Method'

# Not input, prepared for further analysis
weight_list = map(float, weight_string.split(","))  
    # split the string by the commas, and map each string in the resulting list into a float
tickers_list = tickers_string.split(",")  # split the string by the commas into a list of strings
window_days = window * 252                         # convert to days
horizon = horizon_days/252                         # convert to years
start_date = position_date - dateutil.relativedelta.relativedelta(years = 10)      # fetch data starting from start_date 
d={}
for ticker in tickers_list:
    d["{0}".format(ticker)] = web.DataReader(ticker, 'yahoo', start_date, end_date)['Adj Close'].rename(ticker)
df = pd.DataFrame(d).sort_index(ascending = False)  
plot_length = len(df[df.index >= position_date])

# Compute portfolio price
shares = np.round(np.divide(v0 * np.array(weight_list), np.array(df.ix[position_date])))
portfolio = pd.DataFrame({'Price': np.matmul(df, shares)}, index = df.index)

In [20]:
def plot_price_portfolio():
    plot_price(portfolio['Price'], plot_length)

In [21]:
def plot_parameters_portfolio():
    plot_parameters(portfolio['Price'])

In [22]:
def plot_risk_portfolio():
    plot_risk(v0, portfolio['Price'] , VaR_prob, ES_prob, method, window, horizon, plot_length)

In [23]:
##########################################################################################
#################################### OUTPUT TESTING ######################################
##########################################################################################

In [24]:
plot_price_stock()

Loading BokehJS ...

In [30]:
plot_parameters_stock()

Loading BokehJS ...

In [26]:
plot_risk_stock()

Loading BokehJS ...

In [27]:
plot_price_portfolio()

Loading BokehJS ...

In [28]:
plot_parameters_portfolio()

Loading BokehJS ...

In [29]:
plot_risk_portfolio()

Loading BokehJS ...